<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/yahoo_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.Z04s8JXNyR/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Z5iBRixwYa/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.mp2DhYhwDx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [6]:
#環境変数設定
TARGET = '食品'
TARGET_URL = 'https://shopping.yahoo.co.jp/category/2498/list?X=4&sc_i=shp_pc_cate-rcmd_rank&b=1&view=grid'

NUM_TO_FETCH = 25
STORES_PER_PAGE = 30
DRIVER_WAIT_TIME = 2

# 各種ファイル保存先
ACCU_LIST_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_hotpepper/accumulated_list_yahoo_food.csv'
SUBMIT_FILE_PATH ='/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_hotpepper/result_list_yahoo_food.xlsx'

#取得データ格納先
result_dict = { 
    '会社名(ストア情報)': [],
    '住所(ストア情報)': [],
    'ストア名':[],
    '運営責任者(問い合わせ情報)':[],
    '電話番号(問い合わせ情報)':[],
    '電話番号(問い合わせ情報)':[],
    'メールアドレス(問い合わせ情報)':[],
}

In [7]:
#標準ライブラリ
import os
import math
import json
import re

#サードパーティライブラリ
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from logging import (
    getLogger, 
    StreamHandler, 
    DEBUG, 
    INFO, 
    Formatter, 
    config,
)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#関数定義
def add_store_url(elements, store_urls, count_skip):
    for a_element in a_elements:
        store_url = a_element.get_attribute('href')
        if store_url not in accu_set:
            store_urls.append(store_url)
        else:
            count_skip += 1
    return store_urls, count_skip

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})

driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

In [9]:
#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_hotpepper/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{TARGET}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{TARGET}：トップページへ移動出来ませんでした: [e]')
      raise

2023-04-18 10:34:30,256 [DEBUG] main: 食品：トップページへ移動しました


In [11]:
#店舗リストの全ページ数を導出
p_element = driver.find_element(By.CSS_SELECTOR,".SearchResultsDisplayOptions_SearchResultsDisplayOptions__count__WBsPf")
num_all_stores = int(p_element.text.replace(',','')[:-1])
num_all_pages = math.ceil(num_all_stores / STORES_PER_PAGE)


# #積み上げリストから既存取得の店舗URLを抽出
# try:
#   df_accu = pd.read_csv(ACCU_LIST_PATH, usecols=[3])
#   accu_list = df_accu.iloc[:, 0].values.tolist()
#   accu_set = set(accu_list)
# except FileNotFoundError:
#   accu_set = set()

# # 目標件数を満たすまでstore_urlを取得
# # 1ページ目
# try:
#   a_elements = driver.find_elements(By.CSS_SELECTOR,".slnName a")
#   store_urls = []
#   count_skip = 0
#   store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip)
#   logger.debug(
#     f'{TARGET}：店舗リスト取得 (取得数:{len(store_urls)}, 重複数：{count_skip})'
#     )
# except  Exception as e:
#   logger.error(f'{TARGET}：店舗リストへ移動出来ませんでした: [e]')
#   raise

# #2ページ目以降
# i = 1
# while len(store_urls) < NUM_TO_FETCH:
#   i += 1
#   page_url = f'{TARGET_URL}{i}'
#   try:
#     if i < num_all_pages:
#       count_skip = 0
#       driver.get(page_url)
#       # 美容院
#       # a_elements = driver.find_elements(By.CSS_SELECTOR,".slnName a")
#       # リラク
#       a_elements = driver.find_elements(By.CSS_SELECTOR,".slcHead a")
#       store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip)
#       logger.debug(
#         f'{TARGET}：店舗リスト{i}取得 (累計:{len(store_urls)}, 重複：{count_skip})'
#       )
#     else:
#       logger.info(f'{TARGET}：取得出来る店舗情報がありません')
#   except Exception as e:
#     logger.error(f'{TARGET}：店舗リスト{i}へ移動出来ませんでした: [e]')

# logger.info(f'{TARGET}：店舗URL取得完了:{len(store_urls)}個')  
# print(store_urls)


218219


In [ ]:
#店舗個別ページにて情報取得
page_num = 0
try:
  for store_url in store_urls:
    try:
        driver.get(store_url)
        page_num += 1
        logger.debug(f'{TARGET}：店舗ページ{page_num}へ移動しました')
    except Exception as e:
        logger.error(f'{TARGET}：店舗ページへ移動出来ませんでした: [e]')
    try:
        #店舗名/住所取得
        store_name = driver.find_element(By.CSS_SELECTOR,".detailTitle").text
        address = driver.find_element(By.CSS_SELECTOR,".w618").text
        if address=="番号を表示":
          td_elements = driver.find_elements(By.CSS_SELECTOR,".slnDataTbl td")
          address = td_elements[1].text
        #電話番号ページ遷移
        #正規表現を使って、URLの?以降を取り除く
        tmp = re.sub(r'\?.*', '', store_url)
        tel_url = f'{tmp}tel/'
        try:
            driver.get(tel_url)
            logger.debug(f'{TARGET}：店舗電話番号ページ{page_num}へ移動しました')
        except Exception as e:
            logger.error(f'{TARGET}：店舗電話番号ページへ移動出来ませんでした: [{e}]')
        #電話番号取得
        phone_number = driver.find_element(By.CSS_SELECTOR, ".fs16").text
        #結果格納
        result_dict['店名'].append(store_name)
        result_dict['電話番号'].append(phone_number)
        result_dict['住所'].append(address)
        result_dict['URL'].append(store_url)
    except Exception as e:
        logger.error(f'{TARGET}：{store_name}の店舗情報取得に失敗しました: {store_url}')
except KeyboardInterrupt:
  logger.info(f'{TARGET}：処理を中断します')
finally:
  df_result = pd.DataFrame(result_dict)
  if not os.path.isfile(SUBMIT_FILE_PATH):
    df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)
  else:
    with pd.ExcelWriter(SUBMIT_FILE_PATH, mode='a', if_sheet_exists='new') as writer:
      df_result.to_excel(writer)
  df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)
  logger.info(f'{TARGET}：実行終了しデータを保存しました')
  df_accu

df_result = pd.DataFrame(result_dict)
df_result

2023-04-16 23:53:09,890 [DEBUG] main: 千葉県：店舗ページ1へ移動しました
2023-04-16 23:53:12,042 [ERROR] main: 千葉県：POLA フェイシャル＆ボディエステサロン千葉店の店舗情報取得に失敗しました: https://beauty.hotpepper.jp/kr/slnH000334179/?cstt=21
2023-04-16 23:53:14,924 [DEBUG] main: 千葉県：店舗ページ2へ移動しました
2023-04-16 23:53:17,141 [ERROR] main: 千葉県：3Dマツエク Hana by Greenlife 千葉店【はなbyグリーンライフ】の店舗情報取得に失敗しました: https://beauty.hotpepper.jp/kr/slnH000351793/?cstt=22
2023-04-16 23:53:20,318 [DEBUG] main: 千葉県：店舗ページ3へ移動しました
2023-04-16 23:53:22,460 [ERROR] main: 千葉県：元祖強圧リンパ専門店　Wizウィズ　千葉中央の店舗情報取得に失敗しました: https://beauty.hotpepper.jp/kr/slnH000167480/?cstt=23
2023-04-16 23:53:24,681 [DEBUG] main: 千葉県：店舗ページ4へ移動しました
2023-04-16 23:53:26,897 [ERROR] main: 千葉県：千葉中央整体院の店舗情報取得に失敗しました: https://beauty.hotpepper.jp/kr/slnH000298131/?cstt=24
2023-04-16 23:53:28,746 [DEBUG] main: 千葉県：店舗ページ5へ移動しました
2023-04-16 23:53:30,836 [ERROR] main: 千葉県：〈ストレッチ＆ボディケア〉Re.Ra.Ku 千葉中央店【リラク】の店舗情報取得に失敗しました: https://beauty.hotpepper.jp/kr/slnH000288853/?cstt=25
2023-04-16 23:53:32,613 [DEBUG] ma

,店名,住所,電話番号,URL


In [ ]:
# # 提出ファイル用出力
# if not os.path.isfile(SUBMIT_FILE_PATH):
#     df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)
# else:
#   with pd.ExcelWriter(SUBMIT_FILE_PATH, mode='a', if_sheet_exists='new') as writer:
#     df_result.to_excel(writer)

# # 積み上げリスト保存用出力
# df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)

In [ ]:
# 積み上げリスト確認
df_accu = pd.read_csv(ACCU_LIST_PATH)
df_accu

,店名,住所,電話番号,URL
0,バーバー・ナカジマ,東京都北区東十条３－１３－７,03-3911-4665,https://beauty.hotpepper.jp/slnH000528166/?cstt=1
1,MuFF 田町店 【マフ】,東京都港区芝５－１－９NHIビル２F,03-6809-4299,https://beauty.hotpepper.jp/slnH000219477/?cstt=2
2,Shelvie by sins (旧：sins produce by Ran 銀座）,東京都中央区銀座７－３－７ブランエスパ銀座１１F ＴＨＥＳＡＬＯＮＳ内 Ｆ区画,03-4400-8182,https://beauty.hotpepper.jp/slnH000589572/?cstt=3
3,ヘアセット専門店 Mature 上野店 【マチュレ】,東京都台東区東上野３－１４－９ FUJIYAビル１F,03-6886-0265,https://beauty.hotpepper.jp/slnH000346341/?cstt=4
4,LAND 【ランド】,東京都港区南青山３－１３－９ ２Ｆ,03-6447-0162,https://beauty.hotpepper.jp/slnH000349126/?cstt=5
...,...,...,...,...
5073,choconte.,千葉県千葉市中央区登戸１－１１－２０,043-243-3500,https://beauty.hotpepper.jp/slnH000481040/?cst...
5074,verite2～髪質改善 ヘッドスパ～京成大久保,千葉県習志野市大久保１-17-13,047-405-2299,https://beauty.hotpepper.jp/slnH000298820/?cst...
5075,Krump 【クランプ】,千葉県松戸市松戸1281 大塚ビル3F,047-369-7614,https://beauty.hotpepper.jp/slnH000382822/?cst...
5076,TELA HAIR+ 木更津2号店【テーラヘアープラス】,千葉県木更津市清見台南１丁目１０－１８,0438-42-1179,https://beauty.hotpepper.jp/slnH000585774/?cst...
